## Ruumiandmete analüüs

__Prof. Evelyn Uuemaa__

Käesoleva skripti abil saate üles laadida oma CSV-faili ja selle põhjal teha karpdiagrammid ning Mann-Whitney U testi. Lõpuks saate tulemuse aruande jaoks pildifailina alla laadida.

### Kuidas seda skripti kasutada

Mine hiirega kõige esimesele kastile ja vajuta "Run" nuppu, et üksus läbi jooksutada ja minna järgmiste plokkide juurde. Kui midagi valesti läheb, saad sa lihtsasti uuesti algusest alustada.

In [1]:
from IPython.display import display
import ipywidgets as widgets

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

%matplotlib inline

ModuleNotFoundError: No module named 'ipywidgets'

Ülejärgmise ploki juures tuleb sul arvutist valida äsja salvestatud CSV-fail. Kui sinu CSV-faili eraldusmärgiks on koma, pole vaja midagi muuta. Kui aga semikoolon, tee muutus järgmises plokis: csv_delimiter = ";".

In [ ]:
upload = widgets.FileUpload( accept='.csv', multiple=False )

csv_delimiter = ","
text_encoding = "utf8"

In [ ]:
upload

Kui oled faili oma arvutist valinud, peaks üleslaadimise lahter näitama "Upload (1)", mis tähendab, et fail on nüüd valitud.

Jooksuta läbi järgmine plokk. Tulemuseks peaks olema ilus tabel, mitte ühes jorus tekst. Kui näed väärtuste vahel semikooloneid (;), tuleb teises plokis teha parandus (csv_delimiter = ";").

In [ ]:
filestream_encoding = "utf8"

t = upload.data[0].decode(encoding=filestream_encoding)

with open('tmp.csv', 'w', encoding=text_encoding) as fh:
    fh.write(t)

df = pd.read_csv('tmp.csv', encoding=text_encoding, sep=csv_delimiter)

df.head(10)

Nüüd on andmed loetud andmeraami *(dataframe)* formaati ja me saame arvutada statistikuid ning teha jooniseid. Muuda järgmises plokis muutuja "layer" väärtust, kui sul on see veerg mõne teise nimega kui "layer". Seejärel jooksuta järgmist plokki, et saada karpvurrdiagramm.

In [ ]:
classes = "layer"

values = "DISTANCE"

In [ ]:
df['values_prep'] = pd.to_numeric(df[values], errors='coerce')

df = df.rename(columns={values : f"{values}_"})
df = df.rename(columns={'values_prep' : values})

fig, ax = plt.subplots(figsize=(5,7))

sns.boxplot(x=classes, y=values, data=df.dropna(subset=[classes, values]), palette="Spectral")
plt.xticks(
    rotation=0, 
    horizontalalignment='center',
    fontweight='light',
    fontsize='x-large', 
)
plt.xlabel(classes, fontsize='x-large')
plt.yticks(
    rotation=0, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large',
)
plt.ylabel(values, fontsize='x-large')

plt.title(f" {values} box-plots", fontsize=20)

plt.savefig('boxplots.png', bbox_inches='tight', transparent=False)
plt.show()

[lae joonis alla siit](boxplots.png)

Alternatiivina võid ka pildi lihtsalt otse kopeerida, vajutades hiire parema klõpsuga pildil.

**Karpdiagrammide tõlgendamine**
 “karbi” keskjoon näitab mediaani, karbi ülemine ja alumine serv märgib 75% ja 25% protsentiile (1. ja 3. kvartiili), püstjoonte äärmusotsad (vuntsid) markeerivad väljaspoole kvartiilihaaret x 1,5 jäävaid vaatlusi, mida nimetatakse erinditeks (outlier). (Kvartiilihaare on inglise keeles Interquartile range, IQR).

 Võrdle omavahel tegelike maastikutulekahjude ja juhuslike punktide kaugus teest mediaanväärtusi ja tee selle põhjal järeldus, kas maastikutulekahjud on juhusliku või mitte juhusliku paigutusega teede suhtes.


# Mann-Whitney U test

Mann-Whitney U test on mitteparameetriline test, mida kasutatakse kahe rühma võrdlemiseks juhul kui rühmad ei ole normaaljaotusega ja/või vaatlusandmeid on vähe. Valimi suurus peaks olema vähemalt 20 ning võrreldavad muutujad  peaksid olema üksteisest sõltumatud. 

In [ ]:
from scipy.stats import mannwhitneyu

if not len(df[classes].unique()) == 2:
    print("Hoiatus! Vajalik on kaks klassi")

group1 = df.loc[df[classes] == df[classes].unique()[0]][values]
group2 = df.loc[df[classes] == df[classes].unique()[1]][values]

#perform the Mann-Whitney U test
result = mannwhitneyu(group1, group2, alternative='two-sided')
print(f"U statistik {result.statistic}")
print(f"P-väärtus {result.pvalue}")

**Tulemuste tõlgendamine**

Nullhüpotees: kahe rühma jaotus on sarnane

Sisukas hüpotees: kahe rühma jaotus on erinev

Kui p-väärtus ei ole väiksem kui 0,05, siis me ei saa nullhüpoteesi ümber lükata. Seega kahe rühma erinevus on statistiliselt oluline, kui p < 0,05.